In [3]:
import rasterio
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
from tqdm import tqdm
import os
pd.set_option('display.max_columns', None)

In [2]:
# Function to get raster info and pixel count
def get_raster_info(raster_path):
    with rasterio.open(raster_path) as src:
        total_pixels = src.width * src.height
        valid_pixels = np.sum(src.read(1) != src.nodata)
        return {
            "Total pixels": total_pixels,
            "Valid pixels": valid_pixels,
            "NoData pixels": total_pixels - valid_pixels,
            "Width": src.width,
            "Height": src.height,
        }

In [3]:
def create_base_gdf(raster_path, file_name):
    with rasterio.open(raster_path + file_name) as src:
        data = src.read(1)
        transform = src.transform
        crs = src.crs
        
        # Create a grid of all pixel centroids
        rows, cols = np.meshgrid(range(src.height), range(src.width), indexing='ij')
        xs, ys = rasterio.transform.xy(transform, rows.ravel(), cols.ravel(), offset='center')
        
        # Create points for non-NoData values
        mask = data.ravel() != src.nodata
        points = [Point(x, y) for x, y in zip(np.array(xs)[mask], np.array(ys)[mask])]
        values = data.ravel()[mask]
        
        food_name = file_name.split('_')[1]
        
    return gpd.GeoDataFrame({'geometry': points, food_name: values}, crs=crs)

In [4]:
def sample_raster(gdf, raster_path, file_name):
    food_name = file_name.split('_')[1]
    with rasterio.open(raster_path + file_name) as src:
        # Sample the raster at the GeoDataFrame's point locations
        coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
        gdf[food_name] = [x[0] for x in src.sample(coords)]
    return gdf

In [9]:
def process_geotiffs(raster_path, file_list):
    # Create base GeoDataFrame from the first file
    base_gdf = create_base_gdf(raster_path, file_list[0])
    print(f"Created base GeoDataFrame from {file_list[0]}")
    
    # Sample remaining files
    for file in tqdm(file_list[1:], desc="Processing files"):
        base_gdf = sample_raster(base_gdf, raster_path, file)
    
    return base_gdf

In [7]:
# use os to list all filenames  in /Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/2020_gadm_foodgrouprasters/fg2/
raster_path = '/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/2020_gadm_foodgrouprasters/fg2/'
file_list = os.listdir(raster_path)
file_list = [x for x in file_list if not x.startswith('.')]
file_list

['2020_Vegetables_tonnage_gadm.tif',
 '2020_Sweetners_tonnage_gadm.tif',
 '2020_Vegetable-Oils_tonnage_gadm.tif',
 '2020_Treenuts_tonnage_gadm.tif',
 '2020_Stimulants_tonnage_gadm.tif',
 '2020_Spices_tonnage_gadm.tif',
 '2020_Sugar-Crops_tonnage_gadm.tif',
 '2020_Milk-Excluding-Butter_tonnage_gadm.tif',
 '2020_Oilcrops_tonnage_gadm.tif',
 '2020_Pulses_tonnage_gadm.tif',
 '2020_Cereals-Excluding-Beer_tonnage_gadm.tif',
 '2020_Aquatic-Products-Other_tonnage_gadm.tif',
 '2020_Offals_tonnage_gadm.tif',
 '2020_Fish-Seafood_tonnage_gadm.tif',
 '2020_Meat_tonnage_gadm.tif',
 '2020_Eggs_tonnage_gadm.tif',
 '2020_Alcoholic-Beverages_tonnage_gadm.tif',
 '2020_Fruits-Excluding-Wine_tonnage_gadm.tif',
 '2020_Starchy-Roots_tonnage_gadm.tif',
 '2020_Animal-Fats_tonnage_gadm.tif']

In [10]:
result_gdf = process_geotiffs(raster_path, file_list)

Created base GeoDataFrame from 2020_Vegetables_tonnage_gadm.tif


Processing files: 100%|██████████| 19/19 [47:03<00:00, 148.60s/it]


In [36]:
result_gdf = result_gdf.replace(0, np.nan)

In [37]:
non_geometry_columns = result_gdf.columns.drop('geometry')
# Keep rows where at least one non-geometry column is not NaN
gdf_cleaned = result_gdf[result_gdf[non_geometry_columns].notna().any(axis=1)]
print(f"Original shape: {result_gdf.shape}")
print(f"Cleaned shape: {gdf_cleaned.shape}")
print(f"Removed {result_gdf.shape[0] - gdf_cleaned.shape[0]} rows")

Original shape: (9331200, 21)
Cleaned shape: (1918519, 21)
Removed 7412681 rows


In [39]:
gdf_cleaned

,geometry,Vegetables,Sweetners,Vegetable-Oils,Treenuts,Stimulants,Spices,Sugar-Crops,Milk-Excluding-Butter,Oilcrops,Pulses,Cereals-Excluding-Beer,Aquatic-Products-Other,Offals,Fish-Seafood,Meat,Eggs,Alcoholic-Beverages,Fruits-Excluding-Wine,Starchy-Roots,Animal-Fats
364100,POINT (-78.29167 82.95833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.350658,NaN,NaN,NaN,NaN,0.068162,NaN,1.525191,0.042130,NaN,NaN,NaN,0.467383
368396,POINT (-80.29167 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.835334,NaN,NaN,NaN,NaN,0.012665,NaN,0.298146,0.008088,NaN,NaN,NaN,0.115728
368404,POINT (-79.625 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.513004,NaN,NaN,NaN,NaN,0.058110,NaN,1.391083,0.027714,NaN,NaN,NaN,0.623350
368406,POINT (-79.45833 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.714489,NaN,NaN,NaN,NaN,0.084159,NaN,1.900677,0.043892,NaN,NaN,NaN,0.655487
368407,POINT (-79.375 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.600649,NaN,NaN,NaN,NaN,0.106315,NaN,2.385633,0.058007,NaN,NaN,NaN,0.779842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7548383,POINT (-68.04167 -55.625),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.534473,NaN,NaN,NaN,100.398872,0.057795,787.531677,2.815090,0.715552,NaN,NaN,NaN,0.602658
7552709,POINT (-67.54167 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.398872,NaN,787.531677,NaN,NaN,NaN,NaN,NaN,NaN
7552710,POINT (-67.45833 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.398872,NaN,787.531677,NaN,NaN,NaN,NaN,NaN,NaN
7561348,POINT (-67.625 -55.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.398872,NaN,787.531677,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
gdf_cleaned.to_file('/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-full.geojson', driver='GeoJSON')

# Z-Score Normalization

In [ ]:
# do z-score normalization on each of the food group columns and add the values as a new column with the znormal_ prefix
for column in gdf_cleaned.columns:
    if column != 'geometry':
        gdf_cleaned['znormal_' + column] = (gdf_cleaned[column] - gdf_cleaned[column].mean()) / gdf_cleaned[column].std()

In [42]:
gdf_cleaned

,geometry,Vegetables,Sweetners,Vegetable-Oils,Treenuts,Stimulants,Spices,Sugar-Crops,Milk-Excluding-Butter,Oilcrops,Pulses,Cereals-Excluding-Beer,Aquatic-Products-Other,Offals,Fish-Seafood,Meat,Eggs,Alcoholic-Beverages,Fruits-Excluding-Wine,Starchy-Roots,Animal-Fats,znormal_Vegetables,znormal_Sweetners,znormal_Vegetable-Oils,znormal_Treenuts,znormal_Stimulants,znormal_Spices,znormal_Sugar-Crops,znormal_Milk-Excluding-Butter,znormal_Oilcrops,znormal_Pulses,znormal_Cereals-Excluding-Beer,znormal_Aquatic-Products-Other,znormal_Offals,znormal_Fish-Seafood,znormal_Meat,znormal_Eggs,znormal_Alcoholic-Beverages,znormal_Fruits-Excluding-Wine,znormal_Starchy-Roots,znormal_Animal-Fats
364100,POINT (-78.29167 82.95833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.350658,NaN,NaN,NaN,NaN,0.068162,NaN,1.525191,0.042130,NaN,NaN,NaN,0.467383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.282098,NaN,NaN,NaN,NaN,-0.349388,NaN,-0.248993,-0.169072,NaN,NaN,NaN,-0.275192
368396,POINT (-80.29167 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.835334,NaN,NaN,NaN,NaN,0.012665,NaN,0.298146,0.008088,NaN,NaN,NaN,0.115728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.283542,NaN,NaN,NaN,NaN,-0.351325,NaN,-0.250708,-0.169186,NaN,NaN,NaN,-0.279090
368404,POINT (-79.625 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.513004,NaN,NaN,NaN,NaN,0.058110,NaN,1.391083,0.027714,NaN,NaN,NaN,0.623350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281430,NaN,NaN,NaN,NaN,-0.349739,NaN,-0.249181,-0.169120,NaN,NaN,NaN,-0.273463
368406,POINT (-79.45833 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.714489,NaN,NaN,NaN,NaN,0.084159,NaN,1.900677,0.043892,NaN,NaN,NaN,0.655487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281315,NaN,NaN,NaN,NaN,-0.348830,NaN,-0.248469,-0.169066,NaN,NaN,NaN,-0.273107
368407,POINT (-79.375 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.600649,NaN,NaN,NaN,NaN,0.106315,NaN,2.385633,0.058007,NaN,NaN,NaN,0.779842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.280806,NaN,NaN,NaN,NaN,-0.348057,NaN,-0.247791,-0.169019,NaN,NaN,NaN,-0.271729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7548383,POINT (-68.04167 -55.625),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.534473,NaN,NaN,NaN,100.398872,0.057795,787.531677,2.815090,0.715552,NaN,NaN,NaN,0.602658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281418,NaN,NaN,NaN,-0.205162,-0.349750,-0.14292,-0.247191,-0.166829,NaN,NaN,NaN,-0.273692
7552709,POINT (-67.54167 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.398872,NaN,787.531677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN
7552710,POINT (-67.45833 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.398872,NaN,787.531677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN
7561348,POINT (-67.625 -55.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.398872,NaN,787.531677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
gdf_cleaned.columns = gdf_cleaned.columns.str.lower()
# for all the columns that begin with znormal_ create a column that is the column name of the max value
max_columns = [col for col in gdf_cleaned.columns if 'znormal_' in col]
gdf_cleaned['max_food_group'] = gdf_cleaned[max_columns].idxmax(axis=1).str.replace('znormal_', '')

In [48]:
gdf_cleaned.to_file('/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-normal.geojson', driver='GeoJSON')

In [75]:
# drop any columns that are not znormal_ or max_food_group
gdf_normalized = gdf_cleaned.drop(columns=[col for col in gdf_cleaned.columns if 'znormal_' not in col and col != 'max_food_group' and col != 'geometry'])
gdf_normalized

,geometry,znormal_vegetables,znormal_sweetners,znormal_vegetable-oils,znormal_treenuts,znormal_stimulants,znormal_spices,znormal_sugar-crops,znormal_milk-excluding-butter,znormal_oilcrops,znormal_pulses,znormal_cereals-excluding-beer,znormal_aquatic-products-other,znormal_offals,znormal_fish-seafood,znormal_meat,znormal_eggs,znormal_alcoholic-beverages,znormal_fruits-excluding-wine,znormal_starchy-roots,znormal_animal-fats,max_food_group
364100,POINT (-78.29167 82.95833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.282098,NaN,NaN,NaN,NaN,-0.349388,NaN,-0.248993,-0.169072,NaN,NaN,NaN,-0.275192,eggs
368396,POINT (-80.29167 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.283542,NaN,NaN,NaN,NaN,-0.351325,NaN,-0.250708,-0.169186,NaN,NaN,NaN,-0.279090,eggs
368404,POINT (-79.625 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281430,NaN,NaN,NaN,NaN,-0.349739,NaN,-0.249181,-0.169120,NaN,NaN,NaN,-0.273463,eggs
368406,POINT (-79.45833 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281315,NaN,NaN,NaN,NaN,-0.348830,NaN,-0.248469,-0.169066,NaN,NaN,NaN,-0.273107,eggs
368407,POINT (-79.375 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.280806,NaN,NaN,NaN,NaN,-0.348057,NaN,-0.247791,-0.169019,NaN,NaN,NaN,-0.271729,eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7548383,POINT (-68.04167 -55.625),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281418,NaN,NaN,NaN,-0.205162,-0.349750,-0.14292,-0.247191,-0.166829,NaN,NaN,NaN,-0.273692,fish-seafood
7552709,POINT (-67.54167 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood
7552710,POINT (-67.45833 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood
7561348,POINT (-67.625 -55.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood


In [76]:
gdf_normalized_shrunk = gdf_normalized.copy()
znormal_columns = [col for col in gdf_normalized_shrunk.columns if col.startswith('znormal')]
gdf_normalized_shrunk[znormal_columns] = gdf_normalized_shrunk[znormal_columns]*10000
gdf_normalized_shrunk[znormal_columns] = gdf_normalized_shrunk[znormal_columns].round(0)
gdf_normalized_shrunk

,geometry,znormal_vegetables,znormal_sweetners,znormal_vegetable-oils,znormal_treenuts,znormal_stimulants,znormal_spices,znormal_sugar-crops,znormal_milk-excluding-butter,znormal_oilcrops,znormal_pulses,znormal_cereals-excluding-beer,znormal_aquatic-products-other,znormal_offals,znormal_fish-seafood,znormal_meat,znormal_eggs,znormal_alcoholic-beverages,znormal_fruits-excluding-wine,znormal_starchy-roots,znormal_animal-fats,max_food_group
364100,POINT (-78.29167 82.95833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2821.0,NaN,NaN,NaN,NaN,-3494.0,NaN,-2490.0,-1691.0,NaN,NaN,NaN,-2752.0,eggs
368396,POINT (-80.29167 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2835.0,NaN,NaN,NaN,NaN,-3513.0,NaN,-2507.0,-1692.0,NaN,NaN,NaN,-2791.0,eggs
368404,POINT (-79.625 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2814.0,NaN,NaN,NaN,NaN,-3497.0,NaN,-2492.0,-1691.0,NaN,NaN,NaN,-2735.0,eggs
368406,POINT (-79.45833 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2813.0,NaN,NaN,NaN,NaN,-3488.0,NaN,-2485.0,-1691.0,NaN,NaN,NaN,-2731.0,eggs
368407,POINT (-79.375 82.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2808.0,NaN,NaN,NaN,NaN,-3481.0,NaN,-2478.0,-1690.0,NaN,NaN,NaN,-2717.0,eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7548383,POINT (-68.04167 -55.625),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2814.0,NaN,NaN,NaN,-2052.0,-3497.0,-1429.0,-2472.0,-1668.0,NaN,NaN,NaN,-2737.0,fish-seafood
7552709,POINT (-67.54167 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2052.0,NaN,-1429.0,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood
7552710,POINT (-67.45833 -55.70833),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2052.0,NaN,-1429.0,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood
7561348,POINT (-67.625 -55.875),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2052.0,NaN,-1429.0,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood


In [73]:
# convert the znormal columns to integers
gdf_normalized_shrunk.to_file('/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-normal-shrunk-int.geojson', driver='GeoJSON')

In [80]:
# whats the min max value of each column
gdf_normalized_shrunk.describe()

,znormal_vegetables,znormal_sweetners,znormal_vegetable-oils,znormal_treenuts,znormal_stimulants,znormal_spices,znormal_sugar-crops,znormal_milk-excluding-butter,znormal_oilcrops,znormal_pulses,znormal_cereals-excluding-beer,znormal_aquatic-products-other,znormal_offals,znormal_fish-seafood,znormal_meat,znormal_eggs,znormal_alcoholic-beverages,znormal_fruits-excluding-wine,znormal_starchy-roots,znormal_animal-fats,max_food_group_value
count,8.810490e+05,0.0,857526.000000,7.297300e+05,423885.000000,4.589330e+05,551357.000000,1.839434e+06,8.402660e+05,7.993540e+05,8.931080e+05,109754.000000,1.794987e+06,142122.000000,1.892583e+06,1.744120e+06,6.587500e+05,8.884670e+05,8.472910e+05,1.864141e+06,1.918519e+06
mean,8.780442e-03,NaN,0.008420,1.966481e-03,0.010210,-3.845877e-03,0.015928,9.301775e-04,4.811571e-03,1.178452e-02,-3.332184e-03,0.096179,3.042919e-03,0.078412,-4.262957e-03,8.267779e-04,-2.012903e-03,5.791999e-03,2.467865e-03,1.485939e-04,5.795376e+03
std,9.999998e+03,NaN,9999.998047,1.000000e+04,9999.998047,1.000000e+04,9999.998047,1.000000e+04,1.000000e+04,9.999998e+03,1.000000e+04,9999.918945,9.999999e+03,9999.976562,1.000000e+04,1.000000e+04,1.000000e+04,9.999998e+03,9.999999e+03,1.000000e+04,2.433680e+04
min,-2.258000e+03,NaN,-2606.000000,-1.274000e+03,-3457.000000,-1.332000e+03,-1540.000000,-2.840000e+03,-2.333000e+03,-2.030000e+03,-4.725000e+03,-2963.000000,-3.518000e+03,-3938.000000,-2.511000e+03,-1.692000e+03,-1.117000e+03,-2.258000e+03,-2.399000e+03,-2.804000e+03,-3.900000e+03
25%,-2.218000e+03,NaN,-2560.000000,-1.260000e+03,-3453.000000,-1.315000e+03,-1529.000000,-2.796000e+03,-2.302000e+03,-1.985000e+03,-4.602000e+03,-2960.000000,-3.444000e+03,-3318.000000,-2.461000e+03,-1.690000e+03,-1.090000e+03,-2.214000e+03,-2.350000e+03,-2.756000e+03,-1.689000e+03
50%,-2.041000e+03,NaN,-2229.000000,-1.171000e+03,-3418.000000,-1.204000e+03,-1449.000000,-2.541000e+03,-2.085000e+03,-1.766000e+03,-3.679000e+03,-2945.000000,-2.978000e+03,-2769.000000,-2.261000e+03,-1.674000e+03,-9.300000e+02,-1.965000e+03,-2.071000e+03,-2.552000e+03,-1.206000e+03
75%,-1.141000e+03,NaN,-823.000000,-5.640000e+02,-3154.000000,-3.760000e+02,-1022.000000,-1.123000e+03,-1.084000e+03,-8.910000e+02,1.600000e+01,-2621.000000,-5.090000e+02,-1280.000000,-1.092000e+03,-1.447000e+03,-3.090000e+02,-4.900000e+02,-9.390000e+02,-1.136000e+03,3.527000e+03
max,1.420201e+06,NaN,570167.000000,2.790431e+06,117351.000000,4.464816e+06,643686.000000,1.088006e+06,1.119617e+06,4.428434e+06,1.514975e+06,53567.000000,1.218791e+06,438562.000000,9.846770e+05,1.725096e+06,4.786238e+06,5.846762e+06,1.650443e+06,2.535966e+06,5.846762e+06


In [77]:
# This doesn't run here, but adding for documentation on a possible way to remove null values from the geojson
# !jq ' .features |= map( .properties |= with_entries( select(.value != null) ) ) ' food-group-2-normal-shrunk-int.geojson > food-group-2-normal-shrunk-int-nonull.geojson

In [78]:
# create a column that is the value of the max znormal column
gdf_normalized_shrunk['max_food_group_value'] = gdf_normalized_shrunk[max_columns].max(axis=1)
gdf_normalized_tiny = gdf_normalized_shrunk.drop(columns=[col for col in gdf_normalized_shrunk.columns if 'znormal_' in col])
gdf_normalized_tiny

,geometry,max_food_group,max_food_group_value
364100,POINT (-78.29167 82.95833),eggs,-1691.0
368396,POINT (-80.29167 82.875),eggs,-1692.0
368404,POINT (-79.625 82.875),eggs,-1691.0
368406,POINT (-79.45833 82.875),eggs,-1691.0
368407,POINT (-79.375 82.875),eggs,-1690.0
...,...,...,...
7548383,POINT (-68.04167 -55.625),fish-seafood,-1429.0
7552709,POINT (-67.54167 -55.70833),fish-seafood,-1429.0
7552710,POINT (-67.45833 -55.70833),fish-seafood,-1429.0
7561348,POINT (-67.625 -55.875),fish-seafood,-1429.0


In [79]:
gdf_normalized_tiny.to_file('/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-max.geojson', driver='GeoJSON')

In [1]:
#Tiling Script:
!tippecanoe -zg -o z-score-tiles.mbtiles --drop-fraction-as-needed --extend-zooms-if-still-dropping /Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-max.geojson

# Mass Based Normalization

In [4]:
# read '/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-normal.geojson' as geodataframe
all_crops = gpd.read_file('/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-normal.geojson')

In [5]:
all_crops

,vegetables,sweetners,vegetable-oils,treenuts,stimulants,spices,sugar-crops,milk-excluding-butter,oilcrops,pulses,cereals-excluding-beer,aquatic-products-other,offals,fish-seafood,meat,eggs,alcoholic-beverages,fruits-excluding-wine,starchy-roots,animal-fats,znormal_vegetables,znormal_sweetners,znormal_vegetable-oils,znormal_treenuts,znormal_stimulants,znormal_spices,znormal_sugar-crops,znormal_milk-excluding-butter,znormal_oilcrops,znormal_pulses,znormal_cereals-excluding-beer,znormal_aquatic-products-other,znormal_offals,znormal_fish-seafood,znormal_meat,znormal_eggs,znormal_alcoholic-beverages,znormal_fruits-excluding-wine,znormal_starchy-roots,znormal_animal-fats,max_food_group,geometry
0,NaN,None,NaN,NaN,NaN,NaN,NaN,3.350658,NaN,NaN,NaN,NaN,0.068162,NaN,1.525191,0.042130,NaN,NaN,NaN,0.467383,NaN,None,NaN,NaN,NaN,NaN,NaN,-0.282098,NaN,NaN,NaN,NaN,-0.349388,NaN,-0.248993,-0.169072,NaN,NaN,NaN,-0.275192,eggs,POINT (-78.29167 82.95833)
1,NaN,None,NaN,NaN,NaN,NaN,NaN,0.835334,NaN,NaN,NaN,NaN,0.012665,NaN,0.298146,0.008088,NaN,NaN,NaN,0.115728,NaN,None,NaN,NaN,NaN,NaN,NaN,-0.283542,NaN,NaN,NaN,NaN,-0.351325,NaN,-0.250708,-0.169186,NaN,NaN,NaN,-0.279090,eggs,POINT (-80.29167 82.875)
2,NaN,None,NaN,NaN,NaN,NaN,NaN,4.513004,NaN,NaN,NaN,NaN,0.058110,NaN,1.391083,0.027714,NaN,NaN,NaN,0.623350,NaN,None,NaN,NaN,NaN,NaN,NaN,-0.281430,NaN,NaN,NaN,NaN,-0.349739,NaN,-0.249181,-0.169120,NaN,NaN,NaN,-0.273463,eggs,POINT (-79.625 82.875)
3,NaN,None,NaN,NaN,NaN,NaN,NaN,4.714489,NaN,NaN,NaN,NaN,0.084159,NaN,1.900677,0.043892,NaN,NaN,NaN,0.655487,NaN,None,NaN,NaN,NaN,NaN,NaN,-0.281315,NaN,NaN,NaN,NaN,-0.348830,NaN,-0.248469,-0.169066,NaN,NaN,NaN,-0.273107,eggs,POINT (-79.45833 82.875)
4,NaN,None,NaN,NaN,NaN,NaN,NaN,5.600649,NaN,NaN,NaN,NaN,0.106315,NaN,2.385633,0.058007,NaN,NaN,NaN,0.779842,NaN,None,NaN,NaN,NaN,NaN,NaN,-0.280806,NaN,NaN,NaN,NaN,-0.348057,NaN,-0.247791,-0.169019,NaN,NaN,NaN,-0.271729,eggs,POINT (-79.375 82.875)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918514,NaN,None,NaN,NaN,NaN,NaN,NaN,4.534473,NaN,NaN,NaN,100.39887,0.057795,787.53168,2.815090,0.715552,NaN,NaN,NaN,0.602658,NaN,None,NaN,NaN,NaN,NaN,NaN,-0.281418,NaN,NaN,NaN,-0.205162,-0.349750,-0.14292,-0.247191,-0.166829,NaN,NaN,NaN,-0.273692,fish-seafood,POINT (-68.04167 -55.625)
1918515,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.39887,NaN,787.53168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood,POINT (-67.54167 -55.70833)
1918516,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.39887,NaN,787.53168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood,POINT (-67.45833 -55.70833)
1918517,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.39887,NaN,787.53168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.205162,NaN,-0.14292,NaN,NaN,NaN,NaN,NaN,NaN,fish-seafood,POINT (-67.625 -55.875)


In [6]:
# create a max weight column that is the max value of the any column that is not geometry, max_food_group, or max_food_group_value, or starting with z normal
all_crops['max_weight'] = all_crops[[col for col in all_crops.columns if col not in ['geometry', 'max_food_group', 'max_food_group_value'] and not col.startswith('znormal')]].max(axis=1)
all_crops['max_weight_food_group'] = all_crops[[col for col in all_crops.columns if col not in ['geometry', 'max_food_group', 'max_food_group_value'] and not col.startswith('znormal')]].idxmax(axis=1)

In [7]:
max_weight_crops = all_crops.copy()
max_weight_crops = max_weight_crops.drop(columns=[col for col in all_crops.columns if col not in ['geometry', 'max_weight', 'max_weight_food_group']])

In [8]:
max_weight_crops

,geometry,max_weight,max_weight_food_group
0,POINT (-78.29167 82.95833),3.350658,milk-excluding-butter
1,POINT (-80.29167 82.875),0.835334,milk-excluding-butter
2,POINT (-79.625 82.875),4.513004,milk-excluding-butter
3,POINT (-79.45833 82.875),4.714489,milk-excluding-butter
4,POINT (-79.375 82.875),5.600649,milk-excluding-butter
...,...,...,...
1918514,POINT (-68.04167 -55.625),787.53168,fish-seafood
1918515,POINT (-67.54167 -55.70833),787.53168,fish-seafood
1918516,POINT (-67.45833 -55.70833),787.53168,fish-seafood
1918517,POINT (-67.625 -55.875),787.53168,fish-seafood


In [9]:
max_weight_crops['max_weight'].min()

4.5614863e-07

In [10]:
# write to geojson
max_weight_crops.to_file('/Volumes/Samsung 4TB/earthgenome/foodtwin/production-supply/vectorized_production/food-group-2-max-weight.geojson', driver='GeoJSON')